In [3]:

import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json # library to handle JSON files
#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
# import k-means from clustering stage
from sklearn.cluster import KMeans
from sklearn.datasets.samples_generator import make_blobs
#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library
from bs4 import BeautifulSoup
import lxml
import requests
print('Libraries imported.')

Libraries imported.


In [4]:

url="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
 
html_data  = requests.get(url).text

In [5]:
soup = BeautifulSoup(html_data,"html5lib")

In [6]:
# Finding Index of relevant table to be scraped

tables = soup.find_all('table')
for index,table in enumerate(tables):
    if ("M3A" in str(table)):
          table_index = index

print(table_index)

0


In [7]:
tor = pd.DataFrame(columns=["A"])


# Forming a data Frame using for and while loops 

for row in tables[table_index].tbody.find_all("tr"):
    i=0
    while i<9: 
#  As there are 9 colums in table

        col1 = row.find_all("td")
        if (col1 != []):

            a =col1[i].text
            i=i+1           
            tor = tor.append({"A":a}, ignore_index=True)        
tor.head(30)

,A
0,\nM1ANot assigned\n\n
1,\nM2ANot assigned\n\n
2,\nM3ANorth York(Parkwoods)\n\n
3,\nM4ANorth York(Victoria Village)\n\n
4,\nM5ADowntown Toronto(Regent Park / Harbourfro...
5,\nM6ANorth York(Lawrence Manor / Lawrence Heig...
6,\nM7AQueen's Park(Ontario Provincial Governmen...
7,\nM8ANot assigned\n\n
8,\nM9AEtobicoke(Islington Avenue)\n\n
9,\nM1BScarborough(Malvern / Rouge)\n\n


In [8]:
# Removing rows that contain 'Not Assigned'

tor=tor[~tor.A.str.contains('Not assigned')]
tor.tail()

,A
160,\nM8XEtobicoke(The Kingsway / Montgomery Road ...
165,\nM4YDowntown Toronto(Church and Wellesley)\n\n
168,\nM7YEast TorontoBusiness reply mail Processin...
169,\nM8YEtobicoke(Old Mill South / King's Mill Pa...
178,\nM8ZEtobicoke(Mimico NW / The Queensway West ...


In [9]:
#  Removing'\n'

tor["A"]=tor["A"].str.replace("\n","")
tor.head()

,A
2,M3ANorth York(Parkwoods)
3,M4ANorth York(Victoria Village)
4,M5ADowntown Toronto(Regent Park / Harbourfront)
5,M6ANorth York(Lawrence Manor / Lawrence Heights)
6,M7AQueen's Park(Ontario Provincial Government)


In [10]:
# Splitting the dataframe A on the basis of postalcode 

tor['PostalCode']=tor['A'].str.slice(stop=3)
tor['bn']=tor['A'].str.slice(start=3)

tor.drop(columns="A",inplace=True)
tor.head()

,PostalCode,bn
2,M3A,North York(Parkwoods)
3,M4A,North York(Victoria Village)
4,M5A,Downtown Toronto(Regent Park / Harbourfront)
5,M6A,North York(Lawrence Manor / Lawrence Heights)
6,M7A,Queen's Park(Ontario Provincial Government)


In [11]:
# Splitting the borrough and neighbourhood

to=tor["bn"].str.split("(",n=1,expand=True)
to.head()

,0,1
2,North York,Parkwoods)
3,North York,Victoria Village)
4,Downtown Toronto,Regent Park / Harbourfront)
5,North York,Lawrence Manor / Lawrence Heights)
6,Queen's Park,Ontario Provincial Government)


In [12]:

#  Creating new dataframe with desired columns 

data = pd.DataFrame(columns=["PostalCode", "Borough","Neighbourhood"])

data["PostalCode"]=tor["PostalCode"]
data["Borough"]=to[0]
data["Neighbourhood"]=to[1]

data["Neighbourhood"]=data["Neighbourhood"].str.replace(')', '')

data

,PostalCode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Queen's Park,Ontario Provincial Government
8,M9A,Etobicoke,Islington Avenue
9,M1B,Scarborough,Malvern / Rouge
11,M3B,North York,Don MillsNorth
12,M4B,East York,Parkview Hill / Woodbine Gardens
13,M5B,Downtown Toronto,"Garden District, Ryerson"


In [13]:
data.shape

(103, 3)

In [23]:
data.head()

,PostalCode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Queen's Park,Ontario Provincial Government


In [53]:

pip install geocoder

     |████████████████████████████████| 102kB 1.7MB/s ta 0:00:011
Note: you may need to restart the kernel to use updated packages.


In [54]:

url='https://cocl.us/Geospatial_data/Geospatial_Coordinates.csv'
df_coordinates=pd.read_csv(url)
df_coordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [58]:
df = data.rename(columns = {'PostalCode':'Postal Code'})

In [59]:

df = pd.merge(df, df_coordinates, on = 'Postal Code')
df.head()

,Postal Code,Borough,Neighbourhood,Latitude_x,Longitude_x,Latitude_y,Longitude_y
0,M3A,North York,Parkwoods,Not Set,Not Set,43.753259,-79.329656
1,M4A,North York,Victoria Village,Not Set,Not Set,43.725882,-79.315572
2,M5A,Downtown Toronto,Regent Park / Harbourfront,Not Set,Not Set,43.654260,-79.360636
3,M6A,North York,Lawrence Manor / Lawrence Heights,Not Set,Not Set,43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,Not Set,Not Set,43.662301,-79.389494


In [60]:
df.shape

(103, 7)

In [61]:

CLIENT_ID = 'EJB1NCI5KMLFABTKFBACASN1BBSC0IUIFBCL0RELYHUXLVJY' # your Foursquare ID
CLIENT_SECRET = 'TMDZQYITFFEGVXWHL3VTWUXVKI1R1Q5OXNWKAODOHTH3N4GJ' # your Foursquare Secret

VERSION = '20180604'
LIMIT = 30

In [64]:
def getNearbyVenues(names, latitudes, longitudes):
    radius=500
    LIMIT=100
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [66]:
toronto_venues = getNearbyVenues(names=df['Neighbourhood'],
                                   latitudes=df['Latitude_y'],
                                   longitudes=df['Longitude_y']
                                  )

Parkwoods
Victoria Village
Regent Park / Harbourfront
Lawrence Manor / Lawrence Heights
Ontario Provincial Government
Islington Avenue
Malvern / Rouge
Don MillsNorth
Parkview Hill / Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park / Princess Gardens / Martin Grove / Islington / Cloverdale
Rouge Hill / Port Union / Highland Creek
Don MillsSouth(Flemingdon Park
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate / Bloordale Gardens / Old Burnhamthorpe / Markland Wood
Guildwood / Morningside / West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor / Wilson Heights / Downsview North
Thorncliffe Park
Richmond / Adelaide / King
Dufferin / Dovercourt Village
Scarborough Village
Fairview / Henry Farm / Oriole
Northwood Park / York University
The Danforth  East
Harbourfront East / Union Station / Toronto Islands
Little Portugal / Trinity
Kennedy Park / Ionview / East Birchmount Park
B

In [67]:
toronto_venues.groupby('Neighbourhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Agincourt,5,5,5,5,5,5
Alderwood / Long Branch,7,7,7,7,7,7
Bathurst Manor / Wilson Heights / Downsview North,22,22,22,22,22,22
Bayview Village,4,4,4,4,4,4
Bedford Park / Lawrence Manor East,25,25,25,25,25,25
Berczy Park,61,61,61,61,61,61
Birch Cliff / Cliffside West,4,4,4,4,4,4
Brockton / Parkdale Village / Exhibition Place,23,23,23,23,23,23
CN Tower / King and Spadina / Railway Lands / Harbourfront West / Bathurst Quay / South Niagara / Island airport,12,12,12,12,12,12


In [68]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighbourhood,Accessories Store,Adult Boutique,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Korean BBQ Restaurant,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Market,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Moving Target,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Paper / Office Supplies Store,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Truck Stop,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Parkwoods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [69]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()

In [70]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [71]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Breakfast Spot,Latin American Restaurant,Clothing Store,Skating Rink,Lounge,Escape Room,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Diner
1,Alderwood / Long Branch,Pizza Place,Coffee Shop,Gym,Skating Rink,Pub,Sandwich Place,Distribution Center,Dessert Shop,Dim Sum Restaurant,Diner
2,Bathurst Manor / Wilson Heights / Downsview North,Coffee Shop,Bank,Shopping Mall,Pizza Place,Middle Eastern Restaurant,Supermarket,Ice Cream Shop,Sushi Restaurant,Mobile Phone Shop,Restaurant
3,Bayview Village,Café,Bank,Japanese Restaurant,Chinese Restaurant,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant
4,Bedford Park / Lawrence Manor East,Sandwich Place,Italian Restaurant,Coffee Shop,Pharmacy,Juice Bar,Restaurant,Fast Food Restaurant,Japanese Restaurant,Indian Restaurant,Butcher


In [80]:
from sklearn.cluster import KMeans
kclusters = 3
toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

In [81]:
#neighborhoods_venues_sorted.insert(0, 'ClusterLabel', kmeans.labels_)

toronto_merged = df
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')
toronto_merged = toronto_merged.dropna()
toronto_merged

,Postal Code,Borough,Neighbourhood,Latitude_x,Longitude_x,Latitude_y,Longitude_y,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,Not Set,Not Set,43.753259,-79.329656,Park,Food & Drink Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Falafel Restaurant
1,M4A,North York,Victoria Village,Not Set,Not Set,43.725882,-79.315572,Hockey Arena,Pizza Place,Portuguese Restaurant,French Restaurant,Coffee Shop,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Department Store
2,M5A,Downtown Toronto,Regent Park / Harbourfront,Not Set,Not Set,43.654260,-79.360636,Coffee Shop,Bakery,Pub,Park,Breakfast Spot,Restaurant,Café,Theater,Beer Store,Shoe Store
3,M6A,North York,Lawrence Manor / Lawrence Heights,Not Set,Not Set,43.718518,-79.464763,Clothing Store,Accessories Store,Vietnamese Restaurant,Boutique,Furniture / Home Store,Coffee Shop,Electronics Store,Escape Room,Eastern European Restaurant,Dessert Shop
4,M7A,Queen's Park,Ontario Provincial Government,Not Set,Not Set,43.662301,-79.389494,Coffee Shop,Sushi Restaurant,Yoga Studio,Japanese Restaurant,Bank,Bar,Café,Diner,Italian Restaurant,Beer Bar
6,M1B,Scarborough,Malvern / Rouge,Not Set,Not Set,43.806686,-79.194353,Fast Food Restaurant,Dessert Shop,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop
7,M3B,North York,Don MillsNorth,Not Set,Not Set,43.745906,-79.352188,Gym,Japanese Restaurant,Athletics & Sports,Caribbean Restaurant,Café,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Dessert Shop
8,M4B,East York,Parkview Hill / Woodbine Gardens,Not Set,Not Set,43.706397,-79.309937,Pizza Place,Gym / Fitness Center,Athletics & Sports,Flea Market,Gastropub,Bank,Intersection,Pharmacy,Pet Store,Breakfast Spot
9,M5B,Downtown Toronto,"Garden District, Ryerson",Not Set,Not Set,43.657162,-79.378937,Clothing Store,Coffee Shop,Japanese Restaurant,Middle Eastern Restaurant,Café,Cosmetics Shop,Bubble Tea Shop,Bookstore,Hotel,Ramen Restaurant
10,M6B,North York,Glencairn,Not Set,Not Set,43.709577,-79.445073,Park,Bakery,Pizza Place,Asian Restaurant,Japanese Restaurant,Dog Run,Dim Sum Restaurant,Diner,Discount Store,Distribution Center


In [74]:
address = 'Toronto'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

In [77]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude_y'], toronto_merged['Longitude_y'], toronto_merged['Neighbourhood'], toronto_merged['ClusterLabel']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

KeyError: 'ClusterLabel'